In [1]:
from DetectoresEnsamblados import  kComplexDetect, Periodograma_Welch_por_segmento, SpindleDetect,RemDetect,DetectorSW
import mne_bids
import mne
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt


c:\Users\Bruno\OneDrive\Escritorio\Proyecto\.venv\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [ ]:

def salida_predicha(hypno):
    hip = []
    for k, duracion in enumerate(hypno['duration']):
        """
        if n_veces == 0 :
            estado = hypno['trial_type'][k]
            hip.extend([estado] * 1)
        else :
        """
        if duracion == 0:
            n_veces = 1
        else :
            n_veces = int(duracion / 30)
            estado = hypno['trial_type'][k]
            hip.extend([estado] * n_veces)
    
    return hip

In [ ]:
pathSC = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
hipSC = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_events.tsv'

path3 = r'H:\Mi unidad\FormatoBIDS\DatasetSHHS1\sub-00003\ses-001\eeg\sub-00003_ses-001_task-sleep_eeg.edf' #55.0	male
hyp_path3 =  r'H:\Mi unidad\FormatoBIDS\DatasetSHHS1\sub-00003\ses-001\eeg\sub-00003_ses-001_task-sleep_events.tsv'

raw = mne.io.read_raw_edf(path3,preload=True)
#raw.pick_channels(['EEG'])

anotacion = pd.read_csv(hyp_path3, sep = '\t')


In [ ]:
anotacion

# Prueba Kcomplex

In [ ]:
hipno = salida_predicha(anotacion)

In [ ]:
#raw.set_annotations(mne.Annotations(onset=anotacion['onset'],duration=anotacion['duration'], description=anotacion['trial_type']))
raw.set_annotations(mne.Annotations(onset = [30*n for n in np.arange(len(hipno))], duration = np.zeros_like([30*n for n in np.arange(len(hipno))]), description = hipno))### esta es al forma indicada de enviar la anotacion))### esta es al forma indicada de enviar la anotacion

In [ ]:
m = {'NREM1': 1, 'NREM2': 2, 'NREM3': 3, 'REM': 4, 'WAKE': 0}
list(m.values())

In [ ]:
#raw.set_annotations(annot_mne, emit_warning=False)
one_ ,raw_stim = kComplexDetect(raw, 'EEG',{'NREM1': 1, 'NREM2': 2, 'NREM3': 3, 'REM': 4, 'WAKE': 0}, Annotacion_con__duracion  = False) # todavia no anda  usando anotacion_con_duracion = True 

In [ ]:
raw_stim['STIMcomplexK']

In [ ]:
sum(raw_stim['STIMcomplexK'][0][0])
#358

In [ ]:
%matplotlib widget
raw.plot()

## Pruebo otra data

In [ ]:
raw_=r'H:\.shortcut-targets-by-id\1oz7_KTs_0LBRN7CMTRkLDKYI--PGbP_t\CNTR_VMA_NOCHE-BIDS_copy\sub-174\ses-day1\eeg\sub-174_ses-day1_task-sleep_eeg.edf'
hip =  r'H:\.shortcut-targets-by-id\1epHQ10lpZXvKpjB3tVz3XnAwTgWm38El\SUENIO\sub-174\ses-day1\sleep_staging\sub-174_ses-day1_task-sleep_sleepStages.mat'
raw = mne.io.read_raw(raw_, preload=True)
data  =  loadmat(hip)

In [ ]:
anotaciones_event_id = {'NREM1': 1, 'NREM2': 2, 'REM': 4, 'NREM3': 3, 'WAKE': 0}
"""
onsets = [n*30  for n in np.arange(len(annotsleep['trial_type']))]
duractions = [30 for _ in np.arange(len(annotsleep['trial_type']))]
description = annotsleep['trial_type']
annotations = mne.Annotations(onset=onsets, duration=duractions, description=description)

raw.set_annotations(annotations, emit_warning=False)
raw.info

#events_train, _ = mne.events_from_annotations(
#  raw, event_id=anotaciones_event_id, chunk_duration=30.)
"""
hypno = pd.DataFrame()
hypno['onset'] = [n*30  for n in np.arange(data['stageData'][0][0][6].shape[0])] #[n*30  for n in np.arange(len(annotsleep['trial_type']))]
hypno['dur'] =  [30  for n in np.arange(data['stageData'][0][0][6].shape[0])] #[30 for _ in np.arange(len(annotsleep['trial_type']))]
hypno['label'] = data['stageData'][0][0][6].flatten().tolist() #description.map(anotaciones_event_id)
annot_mne = mne.Annotations(onset= [n*30  for n in np.arange(data['stageData'][0][0][6].shape[0])] , duration= [30  for n in np.arange(data['stageData'][0][0][6].shape[0])], description=data['stageData'][0][0][6].flatten().tolist())


In [ ]:
raw_stim = kComplexDetect(raw, 'C4',annot_mne) 

# Prueba Periodograma_Welch_por_segmento

In [ ]:
bandpower = Periodograma_Welch_por_segmento(raw, ['EEG Fpz-Cz','EEG Pz-Oz']) 

In [ ]:
bandpower.shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots( figsize = (15, 7))
x = np.linspace(1, 2650, 2650)


ax.plot(x,bandpower[0,:, 0], label ='delta')
#ax[0].set_title("Delta Plot")
ax.plot(x,bandpower[1,:, 0],label ='theta')
ax.plot(x,bandpower[2,:, 0], label ='alfa')
ax.plot(x,bandpower[3,:, 0],label ='sigma')
ax.plot(x,bandpower[4,:, 0],label ='beta')
ax.legend()
fig.show()


# Prueba REM

In [ ]:
raw_ = r'H:\Mi unidad\FormatoBIDS\DatasetHaaglandenMedischCentrumSleep\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
_= raw = mne.io.read_raw_edf(raw_, preload=True)
raw.set_channel_types({'E1-M2' :'eog', 'E2-M2': 'eog'})

In [3]:
data = raw.get_data(picks= ['E1-M2', 'E2-M2'], units ='uV')

In [5]:
RemDetect(loc =  data[0], roc =  data[1], sf =  raw.info['sfreq'], raw = raw)

Creating RawArray with float64 data, n_channels=1, n_times=6566400
    Range : 0 ... 6566399 =      0.000 ... 25649.996 secs
Ready.


<RawEDF | sub-001_ses-001_task-sleep_eeg.edf, 9 x 6566400 (25650.0 s), ~450.9 MB, data loaded>

In [8]:
np.sum(raw['STIM'][0])

1058.0

# PRUEBA SPINDLE

In [ ]:
raw.info

In [ ]:
raw = SpindleDetect(raw, ['EEG(sec)'])

In [ ]:

np.sum(raw['STIMspindle',:][0])

# PRUEBA SW

In [5]:
raw_2 = r'H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf'
_= raw = mne.io.read_raw_edf(raw_2, preload=True)

Extracting EDF parameters from H:\Mi unidad\FormatoBIDS\DataseSleep-Casset\sub-001\ses-001\eeg\sub-001_ses-001_task-sleep_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


In [6]:
raw.pick_channels(['EEG Fpz-Cz',
 'EEG Pz-Oz'])

<RawEDF | sub-001_ses-001_task-sleep_eeg.edf, 2 x 7950000 (79500.0 s), ~121.3 MB, data loaded>

In [7]:
DetectorSW(raw)

Creating RawArray with float64 data, n_channels=1, n_times=7950000
    Range : 0 ... 7949999 =      0.000 ... 79499.990 secs
Ready.


<RawEDF | sub-001_ses-001_task-sleep_eeg.edf, 3 x 7950000 (79500.0 s), ~182.0 MB, data loaded>

In [8]:
np.sum(raw['STIMSW'][0])

3614.0

In [16]:
def positive_input(func):
    def wrapper(x):
        if x <= 0:
            raise ValueError("Input must be positive")
        return func(x)
    return wrapper

@positive_input
def square(x):
    return x**2


f = positive_input(square)